# Eigenes Environment aufbauen

In [27]:
import gym
import pandas as pd
import numpy as np
import os.path
from functools import reduce

from gym import spaces
import torch


import bs4 as bs
import requests
import yfinance as yf
import datetime
import time
import os
import pandas_datareader as web
import warnings
warnings.filterwarnings('ignore')

class MyDataset():

    def __init__(self):
        self.url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        self.stocks_fname = "sp500_closefull.csv"
        self.start = datetime.datetime(2010, 1, 1)
        self.stop = datetime.datetime.now()
        self.Ntest = 1000
        self.now = time.time()

    def get_train_test(self) -> pd.DataFrame:

        start = self.start
        end = self.stop


        if not os.path.isfile(self.stocks_fname):
          resp = requests.get(self.url)
          soup = bs.BeautifulSoup(resp.text, 'lxml')
          table = soup.find('table', {'class': 'wikitable sortable'})
          tickers = []

          for row in table.findAll('tr')[1:]:
              ticker = row.findAll('td')[0].text
              tickers.append(ticker)

          tickers = [s.replace('\n', '') for s in tickers]
          data = yf.download(tickers, start=start, end=end)
          data['Adj Close'].to_csv(self.stocks_fname)

        df0 = pd.read_csv(self.stocks_fname, index_col=0, parse_dates=True)

        df_spy = yf.download("SPY", start=start, end=end)

        #df_spy = web.DataReader("SPY", "yahoo", start=start, end=end)
        df_spy = df_spy.loc[:, ['Adj Close']]
        df_spy.columns = ['SPY']
        df0 = pd.concat([df0, df_spy], axis=1)


        #df0.dropna(axis=1, how='all', inplace=True)
        df0.dropna(axis=0, how='all', inplace=True)
        print("Dropping columns due to nans > 50%:", df0.loc[:, list((100 * (df0.isnull().sum() / len(df0.index)) > 50))].columns)
        df0 = df0.drop(df0.loc[:, list((100 * (df0.isnull().sum() / len(df0.index)) > 50))].columns, 1)
        df0 = df0.ffill().bfill()

        print("Any columns still contain nans:", df0.isnull().values.any())

        df_returns = pd.DataFrame()
        for name in df0.columns:
          df_returns[name] = np.log(df0[name]).diff()

        #df_returns['SPY_PRICE'] = df0['SPY']
        #df_returns[self.config.stocks_adj_close_names] = df0[self.config.stocks]
        # split into train and test
        df_returns.dropna(axis=0, how='any', inplace=True)


        train_data = df_returns.iloc[:-self.Ntest]
        test_data = df_returns.iloc[-self.Ntest:]


        return train_data, test_data

In [41]:
class EnvConfig():
    def __init__(self):
        self.stocks = ['AAPL', 'MSFT', 'AMZN', 'NFLX', 'XOM', 'JPM', 'T']
        self.initial_cash = 10_000
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

In [44]:
class MyEnv(gym.Env):

    def __init__(self, df_train, df_test, play=False):
        self.df_train = df_train
        self.df_test = df_test
        self.play = play
        if not self.play:
            self.df = self.df_train
        else:
            self.df = self.df_test
        self.config = EnvConfig()
        self.current_idx = 0
        self.stocks = ['AAPL', 'MSFT', 'AMZN', 'NFLX', 'XOM', 'JPM', 'T'] # target stocks
        #self.stocks_adj_close_names = [stock + '_Adj_Close' for stock in self.stocks] # AAPL_Adj_Close so heissen die Spalten bei mir ?

        self.weights = np.full((len(self.config.stocks)), 1 / len(self.config.stocks), dtype=float)

        # cash per stock_values
        self.initial_cash = self.weights * self.config.initial_cash

        self.portfolio_value = 0
        self.stock_values = np.zeros(len(self.stocks))

        self.states = self.df.loc[:, ~self.df.columns.isin(self.stocks)].to_numpy()  # aufpassen, tatsächlich existierende Spaltennamen
        self.rewards = self.df[self.stocks].to_numpy()
        self.n = len(self.states)


    def reset(self):
        self.cash = self.initial_cash
        self.portfolio_value = 0
        self.stock_values = np.zeros(len(self.config.stocks))

        self.current_idx = 0

        next_state = self.states[self.current_idx]
        next_state = np.array(next_state).reshape(1, -1)
        next_state = torch.tensor(next_state).float().to(self.config.device)
        return next_state

    def step(self, actions):
        if self.current_idx >= self.n:
          raise Exception("Episode already done")

        actions = np.array([self.discretized[aa] for aa in actions])

        # actions is an array and contains one action per stock_values
        actions = actions.ravel()

        sharpe = self.get_sharpe(self.play, None)
        self.weights = sharpe.get_weights()
        # ret, vol, sh = sharpe.get_ret_vol_sr(self.weights)

        # print(self.weights)

        current_prices = self.df.iloc[self.current_idx][self.config.stocks_adj_close_names].values
        # current_price = np.dot(self.weights, current_prices) - # todo Fehler?

        self.cash = self.weights * np.sum(self.cash)

        # print(self.cash)
        env_action = np.zeros(len(actions))
        for i, action in enumerate(actions):
          # buy_max = self.cash // (current_price * 1.01)  # 10% provisions
          buy_max = self.cash[i] // current_prices[i]  # ohne 10% provisions

          if action > 0:
              env_action[i] = np.minimum(buy_max, action)
          else:
              # env_action = - np.minimum(self.stock_values, np.abs(action * self.weights)) # todo
              env_action[i] = - np.minimum(self.stock_values[i], np.abs(action))

          self.stock_values[i] += env_action[i]

          if action > 0:
              self.cash[i] -= env_action[i] * current_prices[i]
          else:  # sell
              self.cash[i] += - env_action[i] * current_prices[i]

          provision = np.abs(env_action[i]) * current_prices[i] * .01
          # self.cash -= provisions

          self.provisions[i] += provision

        # state transition
        done = (self.current_idx == self.n - 1)

        self.current_idx += 1

        if not done:

          if action is None:
              raise Exception("NaNs detected!")

          # compute reward
          # print("return", np.sum(env_action * current_price * (np.exp(self.rewards[self.current_idx]) - 1)))
          # print("provisions", np.sum(.01 * np.abs(env_action) * current_price))
          step_reward = np.sum(env_action * current_prices * (np.exp(self.rewards[self.current_idx]) - 1)) - np.sum(.001 * np.abs(env_action) * current_prices)
          next_prices = self.df.iloc[self.current_idx][self.config.stocks_adj_close_names].values
          self.portfolio_value = np.dot(self.stock_values, next_prices)
          '''
          invested_index = np.where(self.stock_values != 0) # invested

          invested_sharpe = self.get_sharpe(self.play, invested_index)
          if invested_sharpe is not None:
              invested_weights = invested_sharpe.get_weights()
              ret, vol, sh = invested_sharpe.get_ret_vol_sr(invested_weights)
          else:
              sh = -1.
              vol = 10

          reward = self.portfolio_value * sh + step_reward * 252

          '''

          reward = self.portfolio_value



          next_state = self.states[self.current_idx]
          kernel = self.states[self.current_idx:self.current_idx+7, 1][::-1]

          print(kernel)
          next_state = np.array(next_state).reshape(1, -1)
          next_state = torch.tensor(next_state).reshape(1, -1).float().to(self.config.device)

        else:
          next_state = None
          reward = 0
          self.portfolio_value = np.dot(self.stock_values, current_prices)

        return next_state, reward, done

In [29]:
dataset = MyDataset()
train, test = dataset.get_train_test()

[*********************100%***********************]  1 of 1 completed
Dropping columns due to nans > 50%: Index(['BF.B', 'BRK.B', 'CARR', 'CDAY', 'CTVA', 'DOW', 'FOX', 'FOXA', 'FTV',
       'HWM', 'IR', 'LW', 'MRNA', 'OGN', 'OTIS', 'UA'],
      dtype='object')
Any columns still contain nans: False


In [46]:
train_env = MyEnv(train,test)
train_env.reset()
train_env.step([0,1,2])

AttributeError: 'MyEnv' object has no attribute 'discretized'

Date
2018-01-29   -0.006085
2018-01-30   -0.029900
2018-01-31   -0.019301
2018-02-01    0.002561
2018-02-02   -0.029065
                ...   
2022-01-10   -0.016772
2022-01-11   -0.009096
2022-01-12    0.023513
2022-01-13    0.015830
2022-01-14   -0.007153
Name: AAP, Length: 1000, dtype: float64